### Packages

In [4]:
import os 
import import_ipynb
from time import sleep
# Working Directory
os.chdir("../data")

# Importing notebooks 
from Import_data import *
from Recup_scams_ex import * 

importing Jupyter notebook from Recup_scams_ex.ipynb


# Idées premières

On dispose de données labellisées comme étant des "scams", des arnaques. 
Ce sont essentiellement des adresses ETH, dont il faut regarder le comportement.
Quels features faire sortir de ces adresses, qui caractériseraient les comportements suspects?

Une fois un modèle de caractérisation d'adresses/contrats suspects choisi, on pourra tenter d'associer les adresses entre elles.

## Caractérisation d'une adresse/contrat suspect

1) Un temps d'inactivté, puis un "boom" d'activité peut-être considéré comme un comportement suspect. On part du principe qu'une adresse ETH est en règle générale idle.

2) Un flux de transactions parallèles et/ou de même montant vers une même ou différentes adresses.

3) Un flux de transactions soudain ("établir un rapport, une proportion"?) qui vide le compte d'un seul coup? Eventuellement un outlier sur la proportion quantité d'ETH (in ou out) / temps en h. 

4) Des transactions vers des oeuvres de charité suspicieuses.

5) Une distance vers un exchange assez court, mais pas direct? (Il faut suivre les transactions qui émanent d'une adresse vers une autre ou plusieurs adresses, de même montant ou de montant divisé, qui mènent à un exchange.)

6) On peut s'attendre a des envois d'ETH entre les adresses scams (on peut tenter de compter ces nombres de transactions en communauté) et des petits (ou gros d'ailleurs) envois vers des adresses non suspectes, peut-être des mules (les compter?). Le regroupement de portefeuilles sera traité en seconde part.

7) Eventuellement, regarder les transactions qui entrent dans les exchanges, de montants assez elevées?

8) Peut etre qu'il se passe aussi un grand nombre de transactions "à blanc"? Petits montants, adresses qui semblent pas liées? IL FAUT REGARDER LES GROS MONTANTS. EPS-Voisinage

IL FAUT REGARDER LE DERNIER NOEUD AVANT L EXCHANGE, CE SERA UN PORTEFEUILLE CONTROLE

9) Obfuscation par l'utilisation de token?

PS: Certains gros utilisateurs font du margin trading, prendre cela en compte par rapport à l'activité des adresses
En gros, ils passent de beaucoup d'ETH, a tres peu d'un coup. Ou bien encore quand il n'y a que des transactions qui viennent d'exchanges.

## Temps 

Gestion de la composante temporelle:
On n'a accès qu'aux adresses si les transactions ont lieu.
Il faut aller chercher suffisamment profondément dans les blocks, afin d'avoir un historique large.

De plus, les transactions se suivent, il faut trouver un moyen de prendre ca en compte.

# Feature engineering

In [5]:
scam_data = get_ethscam_data()

In [29]:
scam_data2 = parse_all_ethscan()

In [116]:
scam_data2.head()

,addresses,precision,balance,txn
0,0xdf9191889649c442836ef55de5036a7b694115b6,Fake_Phishing1431,11371.001874,39
1,0xf6884686a999f5ae6c1af03db92bab9c6d7dc8de,Coinrail Hacker,7446.134011,22
2,0xf2401fa52bcaacfdc67fd7f2b8983ddd15092fa9,Fake_Phishing2753,1164.747959,25
3,0x7F183Cce61eFcABb8d2eb1340f2A94C5EB1Bf8e5,Fake_Phishing618,1084.427728,5
4,0x5411a839780e8b7aceb9353056e9302b81c7f63b,Fake_Phishing2761,646.977000,1


In [50]:
scam_data

,0x2b065809f6ec6df32878bcd26711a0e2bcf59c26,0x11c058c3efbf53939fb6872b09a2b5cf2410a1e2c3f3c867664e43a626d878c0,0x2dfe2e0522cc1f050edcc7a05213bb55bbb36884ec9468fc39eccc013c65b5e4,0x1c6e3348a7ea72ffe6a384e51bd1f36ac1bcb4264f461889a318a3bb2251bf19,0xd0cc2b24980cbcca47ef755da88b220a82291407,0xaa1886de3f70a3ef502ea1379a311c5b4e05f3fd,0x240e125c20a4cc84bd6e7f8d1fd07aff4c06d43d,0x1d60606d8a09b5015d773a80b0c660bb8d91809c,0xadf5b0c2103598fb66a61714152f1d1717d49fe0,0xba83e9ce38b10522e3d6061a12779b7526839eda,...,0x8bba1a1c5e9f72c2ce97a240a7878dd4ae803b61,1gj9tymgjcfsnjprkdcaugemewix2ysbdl,0x0bb4251b8c5e8acb27d6809d77a10aa1cacc5144,qzh7l8neh3v868uzu74qpyat4gjz2rdv0yhjktqcp2,1lptqoigtdmbrlb76g5w7ceja4wshpag9z,0x84a8e201cfce8a10b1e721e1940a78a3326b7c97,qpdjfyy89utdz5ky34exk3s98c9h4ahtwyjgdr9dze,1mx2sjhokgbrnrukj8nsqzur5vlorulhf7,0x8f0e5ef93838ffb9bc2c5afb34c32f55ea97a932,qzuqzn8y2726k2pwx3767zmja9m9wdh5wyukklr6kf
addresses,[0x2b065809f6ec6df32878bcd26711a0e2bcf59c26],[0x11c058c3efbf53939fb6872b09a2b5cf2410a1e2c3f...,[0x11c058c3efbf53939fb6872b09a2b5cf2410a1e2c3f...,[0x11c058c3efbf53939fb6872b09a2b5cf2410a1e2c3f...,[0xd0cc2b24980cbcca47ef755da88b220a82291407],[0xaa1886de3f70a3ef502ea1379a311c5b4e05f3fd],[0x240e125c20a4cc84bd6e7f8d1fd07aff4c06d43d],[0x1d60606d8a09b5015d773a80b0c660bb8d91809c],[0xadf5b0c2103598fb66a61714152f1d1717d49fe0],"[0xba83e9ce38b10522e3d6061a12779b7526839eda, 0...",...,"[1pgkghxgczpbaazu4kfarnmqrnsr3izgwi, 0x8bba1a1...","[1gj9tymgjcfsnjprkdcaugemewix2ysbdl, 0x0bb4251...","[1gj9tymgjcfsnjprkdcaugemewix2ysbdl, 0x0bb4251...","[1gj9tymgjcfsnjprkdcaugemewix2ysbdl, 0x0bb4251...","[1lptqoigtdmbrlb76g5w7ceja4wshpag9z, 0x84a8e20...","[1lptqoigtdmbrlb76g5w7ceja4wshpag9z, 0x84a8e20...","[1lptqoigtdmbrlb76g5w7ceja4wshpag9z, 0x84a8e20...","[1mx2sjhokgbrnrukj8nsqzur5vlorulhf7, 0x8f0e5ef...","[1mx2sjhokgbrnrukj8nsqzur5vlorulhf7, 0x8f0e5ef...","[1mx2sjhokgbrnrukj8nsqzur5vlorulhf7, 0x8f0e5ef..."
category,Phishing,Phishing,Phishing,Phishing,Phishing,Phishing,Fake ICO,Phishing,Fake ICO,Phishing,...,Scamming,Scamming,Scamming,Scamming,Scamming,Scamming,Scamming,Scamming,Scamming,Scamming
coin,ETH,ETH,ETH,ETH,ETH,ETH,ETH,ETH,ETH,ETH,...,ETH,ETH,ETH,ETH,ETH,ETH,ETH,ETH,ETH,ETH
description,slackbot / reddit dms. did not 404.,"did not 404.,MEW Deployed","did not 404.,MEW Deployed","did not 404.,MEW Deployed",NaN,Insertion,fake ICO site w/ fake address.,slackbot / reddit dms. did not 404.,NaN,scambot phishers. did not 404.,...,Trust trading scam site,Fake exchange (same as your-btc.co.uk),Fake exchange (same as your-btc.co.uk),Fake exchange (same as your-btc.co.uk),Fake exchange (same as your-btc.co.uk),Fake exchange (same as your-btc.co.uk),Fake exchange (same as your-btc.co.uk),Fake exchange,Fake exchange,Fake exchange
id,77,81,81,81,2,360,31,88,47,90,...,6612,6604,6604,6604,6605,6605,6605,6548,6548,6548
ip,NaN,69.172.201.153,69.172.201.153,69.172.201.153,NaN,NaN,NaN,NaN,NaN,31.170.160.61,...,185.62.190.207,217.23.6.10,217.23.6.10,217.23.6.10,217.23.6.10,217.23.6.10,217.23.6.10,217.23.6.10,217.23.6.10,217.23.6.10
name,myetherwallet.com.de,myetherwallet.us,myetherwallet.us,myetherwallet.us,myelherwallel.com,myaetherwallet.com,district-0x.io,myetherwallet.com.cm,contribute-status.im,myetherwallet.tech,...,dex-launch.com,gdax.us,gdax.us,gdax.us,hillstrade.net,hillstrade.net,hillstrade.net,your-btc.co.uk,your-btc.co.uk,your-btc.co.uk
nameservers,NaN,"[ns2.uniregistrymarket.link, ns1.uniregistryma...","[ns2.uniregistrymarket.link, ns1.uniregistryma...","[ns2.uniregistrymarket.link, ns1.uniregistryma...",NaN,NaN,NaN,NaN,NaN,"[ns3.hostinger.ru, ns1.hostinger.ru, ns2.hosti...",...,"[ns2.blazingfast.io, ns1.blazingfast.io]","[c358ea2d.bitcoin-dns.hosting, 1a7ea920.bitcoi...","[c358ea2d.bitcoin-dns.hosting, 1a7ea920.bitcoi...","[c358ea2d.bitcoin-dns.hosting, 1a7ea920.bitcoi...","[dns1.registrar-servers.com, dns2.registrar-se...","[dns1.registrar-servers.com, dns2.registrar-se...","[dns1.registrar-servers.com, dns2.registrar-se...","[dns2.registrar-serv

In [91]:
# Indices des adresses scams uniquement liées a l'ETH
ind = scam_data.iloc[2] == "ETH"
ind = np.argwhere(ind)[:,0]
ind

array([   0,    1,    2, ..., 2130, 2131, 2132])

In [96]:
df = pd.DataFrame()

In [102]:
# On récupère l'ensemble du tableau filtré par indices ETH
scam_data.iloc[:,ind]

,0x2b065809f6ec6df32878bcd26711a0e2bcf59c26,0x11c058c3efbf53939fb6872b09a2b5cf2410a1e2c3f3c867664e43a626d878c0,0x2dfe2e0522cc1f050edcc7a05213bb55bbb36884ec9468fc39eccc013c65b5e4,0x1c6e3348a7ea72ffe6a384e51bd1f36ac1bcb4264f461889a318a3bb2251bf19,0xd0cc2b24980cbcca47ef755da88b220a82291407,0xaa1886de3f70a3ef502ea1379a311c5b4e05f3fd,0x240e125c20a4cc84bd6e7f8d1fd07aff4c06d43d,0x1d60606d8a09b5015d773a80b0c660bb8d91809c,0xadf5b0c2103598fb66a61714152f1d1717d49fe0,0xba83e9ce38b10522e3d6061a12779b7526839eda,...,0x8bba1a1c5e9f72c2ce97a240a7878dd4ae803b61,1gj9tymgjcfsnjprkdcaugemewix2ysbdl,0x0bb4251b8c5e8acb27d6809d77a10aa1cacc5144,qzh7l8neh3v868uzu74qpyat4gjz2rdv0yhjktqcp2,1lptqoigtdmbrlb76g5w7ceja4wshpag9z,0x84a8e201cfce8a10b1e721e1940a78a3326b7c97,qpdjfyy89utdz5ky34exk3s98c9h4ahtwyjgdr9dze,1mx2sjhokgbrnrukj8nsqzur5vlorulhf7,0x8f0e5ef93838ffb9bc2c5afb34c32f55ea97a932,qzuqzn8y2726k2pwx3767zmja9m9wdh5wyukklr6kf
addresses,[0x2b065809f6ec6df32878bcd26711a0e2bcf59c26],[0x11c058c3efbf53939fb6872b09a2b5cf2410a1e2c3f...,[0x11c058c3efbf53939fb6872b09a2b5cf2410a1e2c3f...,[0x11c058c3efbf53939fb6872b09a2b5cf2410a1e2c3f...,[0xd0cc2b24980cbcca47ef755da88b220a82291407],[0xaa1886de3f70a3ef502ea1379a311c5b4e05f3fd],[0x240e125c20a4cc84bd6e7f8d1fd07aff4c06d43d],[0x1d60606d8a09b5015d773a80b0c660bb8d91809c],[0xadf5b0c2103598fb66a61714152f1d1717d49fe0],"[0xba83e9ce38b10522e3d6061a12779b7526839eda, 0...",...,"[1pgkghxgczpbaazu4kfarnmqrnsr3izgwi, 0x8bba1a1...","[1gj9tymgjcfsnjprkdcaugemewix2ysbdl, 0x0bb4251...","[1gj9tymgjcfsnjprkdcaugemewix2ysbdl, 0x0bb4251...","[1gj9tymgjcfsnjprkdcaugemewix2ysbdl, 0x0bb4251...","[1lptqoigtdmbrlb76g5w7ceja4wshpag9z, 0x84a8e20...","[1lptqoigtdmbrlb76g5w7ceja4wshpag9z, 0x84a8e20...","[1lptqoigtdmbrlb76g5w7ceja4wshpag9z, 0x84a8e20...","[1mx2sjhokgbrnrukj8nsqzur5vlorulhf7, 0x8f0e5ef...","[1mx2sjhokgbrnrukj8nsqzur5vlorulhf7, 0x8f0e5ef...","[1mx2sjhokgbrnrukj8nsqzur5vlorulhf7, 0x8f0e5ef..."
category,Phishing,Phishing,Phishing,Phishing,Phishing,Phishing,Fake ICO,Phishing,Fake ICO,Phishing,...,Scamming,Scamming,Scamming,Scamming,Scamming,Scamming,Scamming,Scamming,Scamming,Scamming
coin,ETH,ETH,ETH,ETH,ETH,ETH,ETH,ETH,ETH,ETH,...,ETH,ETH,ETH,ETH,ETH,ETH,ETH,ETH,ETH,ETH
description,slackbot / reddit dms. did not 404.,"did not 404.,MEW Deployed","did not 404.,MEW Deployed","did not 404.,MEW Deployed",NaN,Insertion,fake ICO site w/ fake address.,slackbot / reddit dms. did not 404.,NaN,scambot phishers. did not 404.,...,Trust trading scam site,Fake exchange (same as your-btc.co.uk),Fake exchange (same as your-btc.co.uk),Fake exchange (same as your-btc.co.uk),Fake exchange (same as your-btc.co.uk),Fake exchange (same as your-btc.co.uk),Fake exchange (same as your-btc.co.uk),Fake exchange,Fake exchange,Fake exchange
id,77,81,81,81,2,360,31,88,47,90,...,6612,6604,6604,6604,6605,6605,6605,6548,6548,6548
ip,NaN,69.172.201.153,69.172.201.153,69.172.201.153,NaN,NaN,NaN,NaN,NaN,31.170.160.61,...,185.62.190.207,217.23.6.10,217.23.6.10,217.23.6.10,217.23.6.10,217.23.6.10,217.23.6.10,217.23.6.10,217.23.6.10,217.23.6.10
name,myetherwallet.com.de,myetherwallet.us,myetherwallet.us,myetherwallet.us,myelherwallel.com,myaetherwallet.com,district-0x.io,myetherwallet.com.cm,contribute-status.im,myetherwallet.tech,...,dex-launch.com,gdax.us,gdax.us,gdax.us,hillstrade.net,hillstrade.net,hillstrade.net,your-btc.co.uk,your-btc.co.uk,your-btc.co.uk
nameservers,NaN,"[ns2.uniregistrymarket.link, ns1.uniregistryma...","[ns2.uniregistrymarket.link, ns1.uniregistryma...","[ns2.uniregistrymarket.link, ns1.uniregistryma...",NaN,NaN,NaN,NaN,NaN,"[ns3.hostinger.ru, ns1.hostinger.ru, ns2.hosti...",...,"[ns2.blazingfast.io, ns1.blazingfast.io]","[c358ea2d.bitcoin-dns.hosting, 1a7ea920.bitcoi...","[c358ea2d.bitcoin-dns.hosting, 1a7ea920.bitcoi...","[c358ea2d.bitcoin-dns.hosting, 1a7ea920.bitcoi...","[dns1.registrar-servers.com, dns2.registrar-se...","[dns1.registrar-servers.com, dns2.registrar-se...","[dns1.registrar-servers.com, dns2.registrar-se...","[dns2.registrar-serv

In [105]:
addresses = np.array(scam_data.iloc[:,ind].columns)

In [106]:
addresses

array(['0x2b065809f6ec6df32878bcd26711a0e2bcf59c26',
       '0x11c058c3efbf53939fb6872b09a2b5cf2410a1e2c3f3c867664e43a626d878c0',
       '0x2dfe2e0522cc1f050edcc7a05213bb55bbb36884ec9468fc39eccc013c65b5e4',
       ..., '1mx2sjhokgbrnrukj8nsqzur5vlorulhf7',
       '0x8f0e5ef93838ffb9bc2c5afb34c32f55ea97a932',
       'qzuqzn8y2726k2pwx3767zmja9m9wdh5wyukklr6kf'], dtype=object)

In [107]:
prec1 = np.array(scam_data.iloc[1,ind])

In [108]:
prec1

array(['Phishing', 'Phishing', 'Phishing', ..., 'Scamming', 'Scamming',
       'Scamming'], dtype=object)

In [109]:
prec2 = np.array(scam_data.iloc[6,ind])

In [110]:
prec2

array(['myetherwallet.com.de', 'myetherwallet.us', 'myetherwallet.us',
       ..., 'your-btc.co.uk', 'your-btc.co.uk', 'your-btc.co.uk'],
      dtype=object)

In [112]:
prec = []
for i,j in zip(prec1,prec2):
    prec.append(i+" "+j)
prec

['Phishing myetherwallet.com.de',
 'Phishing myetherwallet.us',
 'Phishing myetherwallet.us',
 'Phishing myetherwallet.us',
 'Phishing myelherwallel.com',
 'Phishing myaetherwallet.com',
 'Fake ICO district-0x.io',
 'Phishing myetherwallet.com.cm',
 'Fake ICO contribute-status.im',
 'Phishing myetherwallet.tech',
 'Phishing myetherwallet.com.im',
 'Phishing myetherwallet.com.im',
 'Phishing ethereumchamber.net',
 'Phishing unocoin.fund',
 'Phishing numerai.tech',
 'Phishing OmiseGO.com',
 'Phishing OmiseGO.net',
 'Phishing OmiseGO.com',
 'Phishing myehterwallet.co',
 'Phishing kickico.co',
 'Phishing qtum-token.com',
 'Phishing iconexus.social',
 'Phishing omise-go.io',
 'Phishing enigma-tokens.co',
 'Phishing centra.credit',
 'Phishing kin-klk.com',
 'Scamming the-ether.pro',
 'Phishing monetha.me',
 'Phishing ico-nexus.social',
 'Phishing kirkik.com',
 'Phishing red-pulse.tech',
 'Phishing myethwallet.io',
 'Fake ICO monetha-crowdsale.com',
 'Phishing blockstack.network',
 'Phishing 

In [113]:
df['addresses'] = addresses

In [114]:
df['precision'] = prec

In [117]:
df.head()

,addresses,precision
0,0x2b065809f6ec6df32878bcd26711a0e2bcf59c26,Phishing myetherwallet.com.de
1,0x11c058c3efbf53939fb6872b09a2b5cf2410a1e2c3f3...,Phishing myetherwallet.us
2,0x2dfe2e0522cc1f050edcc7a05213bb55bbb36884ec94...,Phishing myetherwallet.us
3,0x1c6e3348a7ea72ffe6a384e51bd1f36ac1bcb4264f46...,Phishing myetherwallet.us
4,0xd0cc2b24980cbcca47ef755da88b220a82291407,Phishing myelherwallel.com


In [131]:
# Il figure des transactions dans ce tableau, on va les traiter séparément
ind_trans = np.argwhere([len(i) > 42 for i in df.addresses])[:,0]

In [162]:
# Liste des transactions qui figurent dans la liste des addresses a corriger

df.addresses[[len(i) > 42 and i[0:2] == '0x' for i in df.addresses]]
for j in df.addresses[[len(i) > 42 and i[0:2] == '0x' for i in df.addresses]]:
    print(j)

0x11c058c3efbf53939fb6872b09a2b5cf2410a1e2c3f3c867664e43a626d878c0
0x2dfe2e0522cc1f050edcc7a05213bb55bbb36884ec9468fc39eccc013c65b5e4
0x1c6e3348a7ea72ffe6a384e51bd1f36ac1bcb4264f461889a318a3bb2251bf19


In [163]:
urls = []
for j in df.addresses[[len(i) > 42 and i[0:2] == '0x' for i in df.addresses]]:
    urls.append("https://www.etherchain.org/tx/{}".format(j))


['https://www.etherchain.org/tx/0x11c058c3efbf53939fb6872b09a2b5cf2410a1e2c3f3c867664e43a626d878c0', 'https://www.etherchain.org/tx/0x2dfe2e0522cc1f050edcc7a05213bb55bbb36884ec9468fc39eccc013c65b5e4', 'https://www.etherchain.org/tx/0x1c6e3348a7ea72ffe6a384e51bd1f36ac1bcb4264f461889a318a3bb2251bf19']


In [218]:
source = requests.get("https://www.etherchain.org/tx/1c6e3348a7ea72ffe6a384e51bd1f36ac1bcb4264f461889a318a3bb2251bf19").text

In [221]:
soup = BeautifulSoup(source,'lxml')
extract_data = soup.find('div',class_="table-responsive") # on récupère l'ensemble du tableau

In [223]:
extract2 = extract_data.find('table',class_="table table-sm borderless")

In [249]:
extract2.find_all('td')[4].find('a')
string = str(extract2.find_all('td')[4].find('a'))

In [250]:
remove_html_markup(string)

'0xc57f1148855e67763a694F7f2C0e68230AdC686E'

In [229]:
string = str(extract2.find_all('tr')[5].find('td'))

In [231]:
string2 = remove_html_markup(string)

In [238]:
s = str()
for i in range(len(string2)):
    if string2[i] != 'E':
        s+= string2[i]
    else:
        break


In [239]:
s

'4.33864 '

In [240]:
float(s)

4.33864

In [270]:
source = requests.get("https://www.etherchain.org/account/c57f1148855e67763a694f7f2c0e68230adc686e").text
soup = BeautifulSoup(source,'lxml')
extract_data = soup.find('p',class_="mb-4") 

In [272]:
balance = remove_html_markup(str(extract_data))

'0 ETH'

In [286]:
extract = soup.find('ul',class_="list-unstyled list-spaced") 

In [287]:
string = str(extract.find_all('li')[2])

In [289]:
remove_html_markup(string)

'Sent 47 &amp; Received 21 Txns'

In [291]:
[int(s) for s in remove_html_markup(string).split() if s.isdigit()]

[47, 21]

In [292]:
np.sum([int(s) for s in remove_html_markup(string).split() if s.isdigit()])

68

In [297]:
np.argwhere([len(i) > 42 and i[0:2] == '0x' for i in df.addresses])[:,0]

array([1, 2, 3])

In [312]:
scam_data.columns

Index(['0x2b065809f6ec6df32878bcd26711a0e2bcf59c26',
       '0x11c058c3efbf53939fb6872b09a2b5cf2410a1e2c3f3c867664e43a626d878c0',
       '0x2dfe2e0522cc1f050edcc7a05213bb55bbb36884ec9468fc39eccc013c65b5e4',
       '0x1c6e3348a7ea72ffe6a384e51bd1f36ac1bcb4264f461889a318a3bb2251bf19',
       '0xd0cc2b24980cbcca47ef755da88b220a82291407',
       '0xaa1886de3f70a3ef502ea1379a311c5b4e05f3fd',
       '0x240e125c20a4cc84bd6e7f8d1fd07aff4c06d43d',
       '0x1d60606d8a09b5015d773a80b0c660bb8d91809c',
       '0xadf5b0c2103598fb66a61714152f1d1717d49fe0',
       '0xba83e9ce38b10522e3d6061a12779b7526839eda',
       ...
       '0x8bba1a1c5e9f72c2ce97a240a7878dd4ae803b61',
       '1gj9tymgjcfsnjprkdcaugemewix2ysbdl',
       '0x0bb4251b8c5e8acb27d6809d77a10aa1cacc5144',
       'qzh7l8neh3v868uzu74qpyat4gjz2rdv0yhjktqcp2',
       '1lptqoigtdmbrlb76g5w7ceja4wshpag9z',
       '0x84a8e201cfce8a10b1e721e1940a78a3326b7c97',
       'qpdjfyy89utdz5ky34exk3s98c9h4ahtwyjgdr9dze',
       '1mx2sjhokgbrnrukj8nsqzur

In [442]:
def parse_all_ethscam():
    
    # On importe d'abord l'ensemble des données non formatées du site 
    scam_data = get_ethscam_data()
    
    # Indices des adresses scams uniquement liées a l'ETH
    ind = scam_data.iloc[2] == "ETH" 
    ind = np.argwhere(ind)[:,0]
    
    # Création du tableau "propre"
    
    df = pd.DataFrame()
    
    # Récupération des addresses 
    addresses = np.array(scam_data.iloc[:,ind].columns)
    
    # Récupération des descriptifs 
    prec1 = np.array(scam_data.iloc[1,ind])
    prec2 = np.array(scam_data.iloc[6,ind])
    prec = []
    for i,j in zip(prec1,prec2):
        prec.append(i+" "+j)
    
    # On remplit le tableau 
    df['addresses'] = addresses
    df['precision'] = prec
    
    # On retire les addresses non ETH
    
    ind_retirer = np.argwhere([i[0:2] != '0x' for i in df.addresses])[:,0]
    df = df.drop(ind_retirer)
    
    balances = np.zeros(df.shape[0])
    txns = np.zeros(df.shape[0])
    
    """
    
    ADRESSES A CORRIGER D ABORD
    
    """
    
    # Le tableau contient parfois des transactions au lieu d'addresses, on doit corriger
    urls = []
    
    for j in df.addresses[[len(i) > 42 and i[0:2] == '0x' for i in df.addresses]]:
        urls.append("https://www.etherchain.org/tx/{}".format(j))  
    
    balances_corr = []
    txns_corr = []
    
    ind_corr = np.argwhere([len(i) > 42 and i[0:2] == '0x' for i in df.addresses])[:,0]
    j = 0
    
    for url in urls:
        source = requests.get(url).text
        soup = BeautifulSoup(source,'lxml')
        extract_data = soup.find('div',class_="table-responsive") # on récupère l'ensemble du tableau     
        extract2 = extract_data.find('table',class_="table table-sm borderless")
        
        # On souhaite récupérer l'addresse "To" 
        string = str(extract2.find_all('td')[4].find('a'))
        address = remove_html_markup(string)
        
        # Une fois que l'on possède l'addresse liée à la transaction frauduleuse, on tire ses informations
        source = requests.get("https://www.etherchain.org/account/{}".format(address)).text
        soup = BeautifulSoup(source,'lxml')
        extract_data = soup.find('p',class_="mb-4") 
        string = remove_html_markup(str(extract_data))
    
        s = str()
        for i in range(len(string)):
            if string[i] != 'E':
                s+= string[i]
            else:
                break
        
        balance = float(s)
        balances_corr.append(balance)
        
        # On récupère aussi les nombres de transactions 
        
        extract = soup.find('ul',class_="list-unstyled list-spaced") 
        string = str(extract.find_all('li')[2])
        string = remove_html_markup(string)
        txn = np.sum([int(s) for s in string.split() if s.isdigit()])
        txns_corr.append(txn)
        
        ind = ind_corr[j]
        balances[ind] = balance
        txns[ind] = txn
        j += 1 
    
    """
    
    ADRESSES NE NECESSITANT PAS DE CORRECTION
    
    """
    
    ind_no_corr = np.argwhere([len(i) == 42 and i[0:2] == '0x' for i in np.array(scam_data.columns)])[:,0]
    n_iter = 0
    ind_errors = []
    
    for j in ind_no_corr:
        n_iter += 1 
        if (n_iter%5 != 0):
            try:
                
                url = df.iloc[ind_no_corr[j],].addresses
                source = requests.get("https://www.etherchain.org/account/{}".format(url)).text
                soup = BeautifulSoup(source,'lxml')
                extract_data = soup.find('p',class_="mb-4") 
                string = remove_html_markup(str(extract_data))

                s = str()
                for i in range(len(string)):
                    if string[i] != 'E':
                        s+= string[i]
                    else:
                        break
                print(j)
                balance = float(s)
       
                balances[j] = balance
        
        # On récupère aussi les nombres de transactions 
        
                extract = soup.find('ul',class_="list-unstyled list-spaced") 
                string = str(extract.find_all('li')[2])
                string = remove_html_markup(string)
                txn = np.sum([int(s) for s in string.split() if s.isdigit()])
        
                txns[j] = txn
            
            except: 
                print("Error")
                ind_errors.append(j)
        else:
            sleep(7)
            
    df['balance'] = balances
    df['txn'] = txns
    
    return(df,ind_errors)

In [444]:
data_scam,errors = parse_all_ethscam()

0
4
5
6
8
9
10
11
13
14
15
16
18
19
20
21
23
24
25
26
28
29
30
31
33
34
35
36
38
39
40
41
43
44
45
46
48
49
50
51
53
54
55
56
58
59
60
61
63
64
65
66
68
69
70
71
73
74
75
76
78
79
80
81
83
84
85
86
88
89
90
91
93
94
95
96
98
99
100
101
103
104
105
106
108
109
110
111
113
114
115
116
118
119
120
121
123
124
125
126
128
129
130
131
Error
133
134
135
136
138
139
140
141
143
144
145
146
148
149
150
151
153
154
155
156
158
159
160
161
163
164
165
166
168
169
170
171
173
174
175
176
178
179
180
181
183
184
185
186
188
189
190
191
193
194
195
196
198
199
200
Error
201
203
204
205
206
208
209
210
211
213
214
215
216
218
219
220
221
223
224
225
226
228
229
230
231
233
234
235
236
238
239
240
241
243
244
245
246
248
249
250
251
253
Error
254
255
256
258
259
260
261
263
264
265
266
268
269
270
271
273
274
275
276
278
279
280
281
283
284
285
286
288
289
290
291
Error
293
294
295
296
298
299
300
301
303
304
305
307
309
310
311
312
314
315
316
317
319
320
321
322
324
325
326
327
329
330
331
Error
33

In [445]:
data_scam.to_csv('partial_scamlist.csv')

In [446]:
print(errors)

[131, 200, 253, 291, 331, 585, 707, 716, 747, 775, 821, 834, 849, 874, 1047, 1182, 1517, 1558, 1615, 1713, 1777, 1779, 1780, 1781, 1782, 1785, 1786, 1787, 1788, 1790, 1791, 1792, 1793, 1795, 1796, 1797, 1798, 1800, 1801, 1802, 1803, 1805, 1806, 1807, 1809, 1811, 1812, 1813, 1814, 1816, 1817, 1818, 1819, 1821, 1822, 1823, 1824, 1826, 1827, 1828, 1829, 1831, 1832, 1833, 1834, 1836, 1837, 1839, 1840, 1842, 1843, 1845, 1846, 1848, 1849, 1850, 1851, 1853, 1854, 1856, 1857, 1860, 1861, 1862, 1863, 1865, 1866, 1867, 1868, 1870, 1872, 1873, 1874, 1876, 1877, 1878, 1879, 1881, 1882, 1884, 1885, 1893, 1894, 1896, 1897, 1899, 1900, 1901, 1902, 1904, 1905, 1906, 1907, 1911, 1912, 1913, 1914, 1916, 1919, 1921, 1922, 1926, 1927, 1928, 1931, 1934, 1935, 1936, 1937, 1939, 1941, 1942, 1943, 1945, 1946, 1947, 1948, 1950, 1951, 1952, 1954, 1961, 1963, 1965, 1966, 1970, 1973, 1974, 1978, 1982, 1983, 1984, 1985, 1987, 1989, 1991, 1992, 1994, 1995, 1996, 1997, 1999, 2000, 2001, 2002, 2004, 2005, 2006, 2007,

In [451]:
ind = errors 
n_iter = 0
ind_errors = []
    
for j in ind_no_corr:
    n_iter += 1 
    if (n_iter%5 != 0):
        try:
                
            url = df.iloc[ind_no_corr[j],].addresses
            source = requests.get("https://www.etherchain.org/account/{}".format(url)).text
            soup = BeautifulSoup(source,'lxml')
            extract_data = soup.find('p',class_="mb-4") 
            string = remove_html_markup(str(extract_data))

            s = str()
            for i in range(len(string)):
                if string[i] != 'E':
                    s+= string[i]
                else:
                    break
            print(j)
            balance = float(s)
       
            balances[j] = balance

250

In [448]:
data_scam.iloc[131,]

addresses    0xab1c30cd2d9964b3b845ef744e2e4043739430e5
precision                        Phishing bitclaive.com
balance                                               0
txn                                                   0
Name: 131, dtype: object

In [450]:
data_scam.iloc[200,]

addresses    0xb37da5c9179345341fe50d37cfc961ba47c01f9a
precision                          Phishing gatcoins.io
balance                                               0
txn                                                   0
Name: 200, dtype: object

In [ ]:
https://api.etherscan.io/api?module=account&action=balancemulti&address={}&tag=latest&apikey=YourApiKeyToken

In [158]:
def parse_all_ethscam2():

    
    # On importe d'abord l'ensemble des données non formatées du site 
    scam_data = get_ethscam_data()
    
    # Indices des adresses scams uniquement liées a l'ETH
    ind = scam_data.iloc[2] == "ETH" 
    ind = np.argwhere(ind)[:,0]
    
    # Création du tableau "propre"
    
    df = pd.DataFrame()
    
    # Récupération des addresses 
    addresses = np.array(scam_data.iloc[:,ind].columns)
    
    # Récupération des descriptifs 
    prec1 = np.array(scam_data.iloc[1,ind])
    prec2 = np.array(scam_data.iloc[6,ind])
    prec = []
    for i,j in zip(prec1,prec2):
        prec.append(i+" "+j)
    
    # On remplit le tableau 
    df['addresses'] = addresses
    df['precision'] = prec
    
    # On retire les adresses non ETH 
    non_eth_ind = np.argwhere([i[0:2] != '0x' for i in df.addresses])[:,0]
    df = df.drop(non_eth_ind)
    
    # On retire les transactions 
    non_addresses = np.argwhere([len(i) > 42 for i in df.addresses])[:,0]
    df = df.drop(non_addresses)
    
    """
    On complète le tableau avec les balances et les transactions 
    https://api.etherscan.io/api?module=account&action=balancemulti&address={}&tag=latest&apikey=B16PQFQJ9UPYNBP8R6YXF3631EVRTBHUFZ
    EXTRACTION DES BALANCES 
    """
    
    urls = []
    n = 1
    url = str()
    n_iters = np.ceil(df.shape[0]/20) # Le nombre de requetes nécessaires, on ne peut en faire que par batch de 20
    compteur = 0
    
    for add in df.addresses:
        if n%20 != 0:
            url = url + add +',' 
            n += 1
            
            
            
            
        else:
            url = url + add 
            urls.append(url)
            url = str()
            n = 1
            compteur += 1 
            
    if compteur != n_iters:
        url = str()
        for add in df.addresses[(compteur*20)+1:]:
            url = url + add +',' 
            
        url = url[:-1]    
        urls.append(url)
    
    
    # adresse de l'api
    URL = "https://api.etherscan.io/api?module=account&action=balancemulti&address={}&tag=latest&apikey=B16PQFQJ9UPYNBP8R6YXF3631EVRTBHUFZ".format(urls[0])
    r = requests.get(url = URL) 
  
    # extraction du json
    data = r.json() 
    
    res = data['result']
    df_res = pd.DataFrame(data=res)
    df_res['balance'] = df_res['balance'].astype(float)*10**(-18)
    
    """
    EXTRACTION DU NOMBRE DE  TXNS
    
    """
    
    
    
    return(data,df_res,df)
    

In [200]:


# defining the api-endpoint  
API_ENDPOINT = "https://mainnet.infura.io/v3/c6a6f0e057ec465b90b5b85c5cdb5990"
  

data = {"jsonrpc":"2.0","method":"eth_getTransactionCount","params": ["0xab1c30cd2d9964b3b845ef744e2e4043739430e5","latest"],"id":1}
headers = {"Content-Type": "application/json"}

# sending post request and saving response as response object 
r = requests.post(url = API_ENDPOINT, json = data, headers=headers) 
  
# extracting response text  
pastebin_url = r.text 

dict_res = eval(pastebin_url)
res = dict_res['result']
res = int(res,16)

In [203]:
data,df_res,df = parse_all_ethscam2()

In [206]:
data_list = []
for j in df_res.account:
    data_list.append({"jsonrpc":"2.0","method":"eth_getTransactionCount","params": [j,"latest"],"id":1})

In [207]:
data_list[0]

{'jsonrpc': '2.0',
 'method': 'eth_getTransactionCount',
 'params': ['0x2b065809f6ec6df32878bcd26711a0e2bcf59c26', 'latest'],
 'id': 1}

In [208]:
data_list[1]

{'jsonrpc': '2.0',
 'method': 'eth_getTransactionCount',
 'params': ['0xd0cc2b24980cbcca47ef755da88b220a82291407', 'latest'],
 'id': 1}

In [209]:
res_list = []

for j in data_list:
    data = j
    headers = {"Content-Type": "application/json"}
    
    # sending post request and saving response as response object 
    r = requests.post(url = API_ENDPOINT, json = data, headers=headers) 
  
    # extracting response text  
    pastebin_url = r.text 

    dict_res = eval(pastebin_url)
    res = dict_res['result']
    res = int(res,16)
    res_list.append(res)
    

In [211]:
len(data_list)

20